# Part2 completion

In [ ]:
from paoc.constants import ROOT, COOKIE, PAOC
import os
import re
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt; plt.style.use('seaborn-v0_8')

In [ ]:
os.system(f'curl https://adventofcode.com/2023/stats -H "cookie: session={COOKIE}" > {ROOT / "stats.html"}')

with open(ROOT / 'stats.html', 'r') as f:
    lines = f.read().splitlines()

timestamp = datetime.fromtimestamp(os.path.getmtime(ROOT / 'stats.html'))
timestamp = timestamp.utcfromtimestamp(timestamp.timestamp()).strftime("%Y-%m-%d %H:%M:%S")

os.system(f'rm {ROOT / "stats.html"}')

df = pd.DataFrame(columns=['both', 'first_only'])

for line in filter(lambda x: x.startswith('<a href="/2023/day/'), lines):
    hits = re.finditer(r'(?:<[^>]*>|\b(\w+)\b)', line)
    day, both, first_only, *_ = (hit.group() for hit in hits if not hit.group().startswith('<'))
    df.loc[int(day)] = [int(both), int(first_only)]

df = df.sort_index()

df['only_first_p'] = (1 - (df['both'] / (df['both'] + df['first_only']))) * 100

In [ ]:
aoc_bg = '#222222'
aoc_white = '#ffffff'
aoc_green = '#00ff00'
aoc_silver = '#9e9e9e'
aoc_gold = '#ffd700'

fig, ax = plt.subplots()

fig.set_facecolor(aoc_bg)
fig.suptitle('Part 2 difficulty', color=aoc_green, fontsize=16, fontweight='bold')

ax.set_title(f'AoC stats as of {timestamp} [UTC]', color=aoc_white)
ax.set_facecolor(aoc_bg)

ax.set_xlabel('day', color=aoc_green)
ax.set_xticks(df.index)
ax.set_xticklabels(df.index, color=aoc_white)

ax.set_ylabel('% of users that\ndid not complete p2', color=aoc_green)
yticks = range(0, df['only_first_p'].max().astype(int) + 1, 5)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks, color=aoc_white)

ax.bar(df.index, df['only_first_p'], color=aoc_silver, edgecolor=aoc_gold, linewidth=1.5)
ax.grid(alpha=0.1)

fig.tight_layout()
fig.savefig(PAOC / 'misc' / 'part2completion.png', facecolor=fig.get_facecolor(), dpi=500)